# VA simple acoustic scene
This is an example notebook how to create a simple acoustic scene in VA using Python.

#### Prerequisites
If VA Python extension is not installed, add the folder manually.

In [1]:
import sys
sys.path.append( '../../Lib/site-packages' ) # deploy
sys.path.append( '../../dist/Lib/site-packages' ) # dev
import os
print( 'Current working directory:', os.getcwd() )
import va
print( 'Successfully loaded VA Python extension')

Current working directory: C:\Users\berna\Documents\Projects\perSONA\perSONA\python\examples\jupyter
Successfully loaded VA Python extension


In [5]:
import pytta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pytta'

### Connect

In [3]:
if not va.connect( 'localhost' ) :
    raise 'Could not connect to server on localhost, not running?'

Reset VA to clear the scene

In [4]:
va.reset()



Control output gain

In [19]:
va.set_output_gain( 0.25 )

Add the current working directory and any further relative or absolute directories where you want to put resource files. VA provides search paths where to look for any file. From now on, only use relative paths or macros to paths.
> Pathes are relevant on server side, not on a remote client. The files must be available on the computer **where the VA application is running**!

In [20]:
va.add_search_path( os.getcwd() );
va.add_search_path( 'C:\dev\VA\VACore\data' );

### Signal source
Create a signal source from a file and start playback with looping mode

In [6]:
signal_source_id = va.create_signal_source_buffer_from_file( '$(DemoSound)' )

In [7]:
va.set_signal_source_buffer_playback_action_str( signal_source_id, 'play' )
va.set_signal_source_buffer_looping( signal_source_id, False )

### Virtual sound source
Create a virtual sound source with any name and set a position

In [30]:
sound_source_id = va.create_sound_source( 'scene' )
va.set_sound_source_position( sound_source_id, ( -2.5, 1.7, -2.1 ) )
va.set_sound_source_signal_source( sound_source_id, signal_source_id )

### Connect signal and source
Connect the signal source to the virtual sound source

In [31]:
va.set_sound_source_signal_source( sound_source_id, signal_source_id )

### Head-related transfer function / Head-related impulse response
Load an HRIR (time domain representation of an HRTF) as a directivity that will be assigned to a sound receiver (aka listener). See [OpenDAFF](http://www.opendaff.org) for more information.
> We use a macro `DefaultHRIR` here, that is usually available for a VA core.

In [21]:
hrir_id = va.create_directivity_from_file( '$(DefaultHRIR)' )

### Virtual listener
Create a sound receiver with arbitrary name, assign the HRTF/HRIR and set a position.

In [22]:
listener_id = va.create_sound_receiver( 'Bernardo' )
va.set_sound_receiver_position( listener_id, ( 0, 1.7, 0 ) )
va.set_sound_receiver_orientation_vu( listener_id, ( 0, 0, -1, ), ( 0, 1, 0 ) ) #  Default view is to -Z (OpenGL)
va.set_sound_receiver_directivity( listener_id, hrir_id )

In [23]:
va.set_signal_source_buffer_playback_action_str( signal_source_id, 'stop' )

In [23]:
import time
import numpy as np

In [25]:
positions = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
positions = [1, 2, 3, 4 ,5, 5, 5, 7, 1, 2, 12]
for position in positions:
    azimuth = position*(360/len(positions))
    radius = 1
    signal_source_id = va.create_signal_source_buffer_from_file( f'../../../../UFSC_Demos/audio/{position}.wav' )
    sound_source_id = va.create_sound_source( 'Numbers' )
    va.set_sound_source_position( sound_source_id, ( radius*np.sin(azimuth), 1.7, radius*np.cos(azimuth) ) )
    va.set_sound_source_signal_source( sound_source_id, signal_source_id )
    va.set_signal_source_buffer_playback_action_str( signal_source_id, 'play' )
    time.sleep(0.75)

In [86]:
dist     = 2
nlegs       = 200
Tval        = 15
z_start   = -70
z_end     = 50

signal_source_id = va.create_signal_source_buffer_from_file( f'../../../../UFSC_Demos/audio/ambulance_long.wav' )

In [88]:
sound_source_id = va.create_sound_source( 'Ambulance Static' )
va.set_sound_source_position( sound_source_id, (dist, 1.7, -70 ) )
va.set_sound_source_sound_power(sound_source_id, 0.55)
va.set_sound_source_signal_source( sound_source_id, signal_source_id )
va.set_signal_source_buffer_playback_action_str( signal_source_id, 'play' )

In [89]:
va.set_signal_source_buffer_playback_action_str( signal_source_id, 'stop' )

In [ ]:
va.set_signal_source_buffer_looping( signal_source_id, False )

In [87]:
sound_source_id = va.create_sound_source( 'Ambulance Moving' )
va.set_sound_source_signal_source( sound_source_id, signal_source_id )

S1pos_traj = np.linspace(z_start,z_end,nlegs)

for position in range(nlegs):
    if position ==0: 
        print('here')
        va.set_sound_source_position( sound_source_id, (dist, 1.7, S1pos_traj[0] ) )
        time.sleep(1)
        va.set_signal_source_buffer_playback_action_str( signal_source_id, 'play' )
        va.set_signal_source_buffer_looping( signal_source_id, True )
    time.sleep(Tval/nlegs)
    va.set_sound_source_position( sound_source_id, (dist, 1.7, S1pos_traj[position] ) )
    
    if position == nlegs-1:
        va.set_signal_source_buffer_playback_action_str( signal_source_id, 'stop' )
    

here


### Active listener
Set an active listener. This concept is deprecated, but should be used for compatibility until it is removed.

In [ ]:
va.set_active_sound_receiver( listener_id )

### Disconnect

In [ ]:
va.disconnect();